In [28]:
import pandas as pd
import warnings
import re
import tables
warnings.filterwarnings('ignore')

# Lendo planilhas

In [29]:
df_animais_sem_id = pd.read_hdf('df_animais_sem_id.h5', key='df')
df_genealogia_embrapa = pd.read_hdf('df_genealogia_embrapa.h5', key='df')
pedigree = pd.read_csv("pedrigree - Girolando - 17-02-2023.txt", encoding='ISO-8859-1')

# Imprimindo dataframes

In [30]:
df_animais_sem_id

,reb_giro,nomea,COMPRAC,RGVACA,nomep,GSPAI,RGPAI,nomem,GSMAE,RGMAE,...,sexo,cod_prod,cod_pai,cod_mae,reb,id_produto,nome10caracteres,id_produto_x,id_produto_y,AN
0,7461,NANA IMPERADOR BAXTER RF SAO F,5858,6353-AH,IMPERADOR BAXTER VOLTA FRIA,58,1459,JURITI HERBY RF SAO FRANCISCO,58,6877-N,...,F,NaN,NaN,NaN,1291,NaN,NANAIMPERA,NaN,NaN,2015
1,10639,LANAA FIV AFTERSHOCK BON.BELEM,12,9592-AG,MS ATLEES SHT AFTERSHOCK - ET,10,AX129928,JOANE FIV KUBERA,11,ACFG1840,...,F,NaN,NaN,9513273.0,10639-GIRO,NaN,LANAAFIV,NaN,NaN,2014
2,6039,EDILAINE IMPERIO FIV 4303 JARD,12,AADA0352,IMPERIO TE STA EDWIGES,11,RIG163,SERTEC QATAR FABIANA PROGRESS,10,HBB/B-291392,...,F,NaN,4320031.0,NaN,NaN,NaN,EDILAINEIM,NaN,NaN,2013
3,1945,ARMA MERIDIAN DA MBL,12,1025-AN,SULLY HART MERIDIAN-ET,10,AX138576,FAMA,11,JNR167,...,F,NaN,NaN,10621711.0,37634-ABCZ,NaN,ARMAMERIDI,NaN,NaN,2015
4,73119,CELESTE 136 TEATRO FIV COOPERR,12,1239-BF,TEATRO DA SILVANIA,11,EFC383,LAGOS ASTRE QUIMICA 795 TE,10,BX335536,...,F,NaN,3540404.0,NaN,NaN,NaN,CELESTE136,NaN,NaN,2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
575,1945,JPZ QUIRERA SHAMROCK ASTER FIV,12,4032-Z,LADYS-MANOR PL SHAMROCK-ET,10,AX135714,JPZ ASTER FIV,11,JPZ64,...,F,NaN,NaN,11428022.0,37634-ABCZ,NaN,JPZQUIRERA,NaN,NaN,2014
576,2381,DALIA I FIV BRADLEY JM NOVO HO,12,4906-AZ,ROCKALLI BRADLEY,10,AX117720,DALIA NOVO HORIZONTE,11,JMCH76,...,F,NaN,NaN,12866148.0,31300-ABCZ,NaN,DALIAIFIV,NaN,NaN,2017
577,132,QUINTA I COLE AGM TRIUNFO,34,9243-AG,CO-OP TOYSTORY COLE-ET,10,AX129567,QUINTA FIV M,12,0473-O,...,F,NaN,NaN,NaN,31835-ABCZ,NaN,QUINTAICO,NaN,NaN,2015
578,6915,LIA DAS ARABIAS,5858,6125-BR,TURBO DAS ARABIAS,5858,0079,LILIKA DAS ARABIAS,58,4916-AW,...,F,NaN,NaN,NaN,27729-ABCZ,NaN,LIADASARA,NaN,NaN,2019


In [4]:
pedigree.columns

Index(['NOMEVACA', 'RGVACA', 'COMPRAC', 'IDANIMAL', 'DNASC', 'SEXO', 'NOMEPAI',
       'RGPAI', 'GSPAI', 'IDPAI', 'NOMEMAE', 'RGMAE', 'GSMAE', 'IDMAE', 'gs',
       'cgen_a', 'comb'],
      dtype='object')

# Criando dataframe juntando os dois dataframes

In [31]:
df_todas_colunas = pd.merge(df_animais_sem_id, pedigree, on=['IDANIMAL'], how='left', indicator=True, suffixes=('','_drop'))
df_todas_colunas = df_todas_colunas.drop(df_todas_colunas.filter(regex='_drop$').columns, axis=1)
df_todas_colunas = df_todas_colunas[df_todas_colunas['_merge']=='both']
df_todas_colunas

,reb_giro,nomea,COMPRAC,RGVACA,nomep,GSPAI,RGPAI,nomem,GSMAE,RGMAE,...,id_produto_y,AN,NOMEVACA,DNASC,SEXO,NOMEPAI,NOMEMAE,cgen_a,comb,_merge
0,7461,NANA IMPERADOR BAXTER RF SAO F,5858,6353-AH,IMPERADOR BAXTER VOLTA FRIA,58,1459,JURITI HERBY RF SAO FRANCISCO,58,6877-N,...,NaN,2015,NANA IMPERADOR BAXTER RF SAO FRANCISCO,11/03/2015,F,IMPERADOR BAXTER VOLTA FRIA,JURITI HERBY RF SAO FRANCISCO,5/8HOL 3/8GIR,666,both
1,10639,LANAA FIV AFTERSHOCK BON.BELEM,12,9592-AG,MS ATLEES SHT AFTERSHOCK - ET,10,AX129928,JOANE FIV KUBERA,11,ACFG1840,...,NaN,2014,LANCA FIV AFTERSHOCK BON.BELEM,22/12/2014,F,MS ATLEES SHT AFTERSHOCK - ET,JOANE FIV KUBERA,1/2HOL 1/2GIR,519,both
2,6039,EDILAINE IMPERIO FIV 4303 JARD,12,AADA0352,IMPERIO TE STA EDWIGES,11,RIG163,SERTEC QATAR FABIANA PROGRESS,10,HBB/B-291392,...,NaN,2013,EDILAINE IMPERIO FIV 4303 JARDIM DO TURVO,18/10/2013,F,IMPERIO TE STA EDWIGES,SERTEC QATAR FABIANA PROGRESS,1/2HOL 1/2GIR,519,both
3,1945,ARMA MERIDIAN DA MBL,12,1025-AN,SULLY HART MERIDIAN-ET,10,AX138576,FAMA,11,JNR167,...,NaN,2015,ARMA MERIDIAN DA MBL,27/04/2015,F,SULLY HART MERIDIAN-ET,FAMA,1/2HOL 1/2GIR,519,both
4,73119,CELESTE 136 TEATRO FIV COOPERR,12,1239-BF,TEATRO DA SILVANIA,11,EFC383,LAGOS ASTRE QUIMICA 795 TE,10,BX335536,...,NaN,2016,CELESTE 136 TEATRO FIV COOPERRITA,05/02/2016,F,TEATRO DA SILVANIA,LAGOS ASTRE QUIMICA 795 TE,1/2HOL 1/2GIR,519,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
562,1945,JPZ QUIRERA SHAMROCK ASTER FIV,12,4032-Z,LADYS-MANOR PL SHAMROCK-ET,10,AX135714,JPZ ASTER FIV,11,JPZ64,...,NaN,2014,JPZ QUIRERA SHAMROCK ASTER FIV,18/02/2014,F,LADYS-MANOR PL SHAMROCK-ET,JPZ ASTER FIV,1/2HOL 1/2GIR,519,both
563,2381,DALIA I FIV BRADLEY JM NOVO HO,12,4906-AZ,ROCKALLI BRADLEY,10,AX117720,DALIA NOVO HORIZONTE,11,JMCH76,...,NaN,2017,DALIA I FIV BRADLEY JM NOVO HORIZONTE,19/08/2017,F,ROCKALLI BRADLEY,DALIA NOVO HORIZONTE,1/2HOL 1/2GIR,519,both
564,132,QUINTA I COLE AGM TRIUNFO,34,9243-AG,CO-OP TOYSTORY COLE-ET,10,AX129567,QUINTA FIV M,12,0473-O,...,NaN,2015,QUINTA I COLE AGM TRIUNFO,25/04/2015,F,CO-OP TOYSTORY COLE-ET,QUINTA FIV M,3/4HOL 1/4GIR,795,both
565,6915,LIA DAS ARABIAS,5858,6125-BR,TURBO DAS ARABIAS,5858,0079,LILIKA DAS ARABIAS,58,4916-AW,...,NaN,2019,LIA DAS ARABIAS,13/06/2019,F,TURBO DAS ARABIAS,LILIKA DAS ARABIAS,5/8HOL 3/8GIR,666,both


# Criando id_produto

In [32]:
df_todas_colunas['id_produto'] = df_todas_colunas['id_produto'].fillna('GIRO' + df_todas_colunas['IDANIMAL'].astype(str))
df_todas_colunas = df_todas_colunas.drop(columns='_merge')
df_todas_colunas

,reb_giro,nomea,COMPRAC,RGVACA,nomep,GSPAI,RGPAI,nomem,GSMAE,RGMAE,...,id_produto_x,id_produto_y,AN,NOMEVACA,DNASC,SEXO,NOMEPAI,NOMEMAE,cgen_a,comb
0,7461,NANA IMPERADOR BAXTER RF SAO F,5858,6353-AH,IMPERADOR BAXTER VOLTA FRIA,58,1459,JURITI HERBY RF SAO FRANCISCO,58,6877-N,...,NaN,NaN,2015,NANA IMPERADOR BAXTER RF SAO FRANCISCO,11/03/2015,F,IMPERADOR BAXTER VOLTA FRIA,JURITI HERBY RF SAO FRANCISCO,5/8HOL 3/8GIR,666
1,10639,LANAA FIV AFTERSHOCK BON.BELEM,12,9592-AG,MS ATLEES SHT AFTERSHOCK - ET,10,AX129928,JOANE FIV KUBERA,11,ACFG1840,...,NaN,NaN,2014,LANCA FIV AFTERSHOCK BON.BELEM,22/12/2014,F,MS ATLEES SHT AFTERSHOCK - ET,JOANE FIV KUBERA,1/2HOL 1/2GIR,519
2,6039,EDILAINE IMPERIO FIV 4303 JARD,12,AADA0352,IMPERIO TE STA EDWIGES,11,RIG163,SERTEC QATAR FABIANA PROGRESS,10,HBB/B-291392,...,NaN,NaN,2013,EDILAINE IMPERIO FIV 4303 JARDIM DO TURVO,18/10/2013,F,IMPERIO TE STA EDWIGES,SERTEC QATAR FABIANA PROGRESS,1/2HOL 1/2GIR,519
3,1945,ARMA MERIDIAN DA MBL,12,1025-AN,SULLY HART MERIDIAN-ET,10,AX138576,FAMA,11,JNR167,...,NaN,NaN,2015,ARMA MERIDIAN DA MBL,27/04/2015,F,SULLY HART MERIDIAN-ET,FAMA,1/2HOL 1/2GIR,519
4,73119,CELESTE 136 TEATRO FIV COOPERR,12,1239-BF,TEATRO DA SILVANIA,11,EFC383,LAGOS ASTRE QUIMICA 795 TE,10,BX335536,...,NaN,NaN,2016,CELESTE 136 TEATRO FIV COOPERRITA,05/02/2016,F,TEATRO DA SILVANIA,LAGOS ASTRE QUIMICA 795 TE,1/2HOL 1/2GIR,519
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
562,1945,JPZ QUIRERA SHAMROCK ASTER FIV,12,4032-Z,LADYS-MANOR PL SHAMROCK-ET,10,AX135714,JPZ ASTER FIV,11,JPZ64,...,NaN,NaN,2014,JPZ QUIRERA SHAMROCK ASTER FIV,18/02/2014,F,LADYS-MANOR PL SHAMROCK-ET,JPZ ASTER FIV,1/2HOL 1/2GIR,519
563,2381,DALIA I FIV BRADLEY JM NOVO HO,12,4906-AZ,ROCKALLI BRADLEY,10,AX117720,DALIA NOVO HORIZONTE,11,JMCH76,...,NaN,NaN,2017,DALIA I FIV BRADLEY JM NOVO HORIZONTE,19/08/2017,F,ROCKALLI BRADLEY,DALIA NOVO HORIZONTE,1/2HOL 1/2GIR,519
564,132,QUINTA I COLE AGM TRIUNFO,34,9243-AG,CO-OP TOYSTORY COLE-ET,10,AX129567,QUINTA FIV M,12,0473-O,...,NaN,NaN,2015,QUINTA I COLE AGM TRIUNFO,25/04/2015,F,CO-OP TOYSTORY COLE-ET,QUINTA FIV M,3/4HOL 1/4GIR,795
565,6915,LIA DAS ARABIAS,5858,6125-BR,TURBO DAS ARABIAS,5858,0079,LILIKA DAS ARABIAS,58,4916-AW,...,NaN,NaN,2019,LIA DAS ARABIAS,13/06/2019,F,TURBO DAS ARABIAS,LILIKA DAS ARABIAS,5/8HOL 3/8GIR,666


In [7]:
df_genealogia_embrapa

,id_produto,id,abcz,registro,nome,sexo,paternidade,ck_sire,id_touro,id_vaca,...,rebc2_pr,cat,ficha_a,cod_prod,IDANIMAL,origem,registro_pai,registro_mae,AN,nome10caracteres
0,0001,0001,None,None,nan,M,None,None,None,None,...,99999,None,0,0,0,OUTRAS,None,None,1982,nan
1,0003,0003,None,None,nan,M,None,None,None,None,...,99999,None,0,0,0,OUTRAS,None,None,1968,nan
2,0006,0006,None,None,nan,M,None,None,None,None,...,99999,None,0,0,0,OUTRAS,None,None,1967,nan
3,02418-249,02418-249,None,2418,nan,F,None,None,108115,108615,...,99999,None,0,0,0,OUTRAS,A1343,18935,1971,nan
4,03,03,None,None,nan,M,None,None,None,None,...,99999,None,0,0,0,OUTRAS,None,None,1965,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
463543,ZZZ7,ZZZ7,S,ZZZ7,AMBICAO,F,None,None,KCA472,135177,...,11958-ABCZ 1,PO,1351780,3158020,0,ABCGIL,KCA472,KB429,1998,AMBICAO
463544,ZZZ8,ZZZ8,S,ZZZ8,AGASALHO,M,None,None,121118,131985,...,7145-ABCZ 1,PO,0,3247323,0,ABCZ,K830,KA7368,1998,AGASALHO
463545,ZZZ9,ZZZ9,S,ZZZ9,BACABAL,M,None,None,133175,178922,...,7145-ABCZ 1,PO,0,3247322,0,ABCZ,K974,KA9422,1999,BACABAL
463546,ZZZA1,ZZZA1,S,ZZZA1,ANITA,F,None,None,EFC383,JDRB874,...,7145-ABCZ 1,PO,0,9499094,0,ABCZ,EFC383,JDRB874,2007,ANITA


# Juntando df_todas_colunas (idpai) com genealogia_embrapa (idanimal) tabela separada para pedigree

In [51]:
teste = pd.merge(df_todas_colunas, df_genealogia_embrapa[['id_produto', 'IDANIMAL']],left_on=['IDPAI'], right_on=['IDANIMAL'], suffixes=('', '_drop'))
teste = teste.drop_duplicates(subset='IDANIMAL')
teste

,reb_giro,nomea,COMPRAC,RGVACA,nomep,GSPAI,RGPAI,nomem,GSMAE,RGMAE,...,DNASC,SEXO,NOMEPAI,NOMEMAE,cgen_a,comb,id_touro,id_vaca,id_produto_drop,IDANIMAL_drop
0,7461,NANA IMPERADOR BAXTER RF SAO F,5858,6353-AH,IMPERADOR BAXTER VOLTA FRIA,58,1459,JURITI HERBY RF SAO FRANCISCO,58,6877-N,...,11/03/2015,F,IMPERADOR BAXTER VOLTA FRIA,JURITI HERBY RF SAO FRANCISCO,5/8HOL 3/8GIR,666,NaN,NaN,GIRO977401,977401
1,7450,OCULTA IMPERADOR BAXTER SJ LAL,5858,7677-BB,IMPERADOR BAXTER VOLTA FRIA,58,1459,IONA 777 SUPERSTITION SJ LALU,58,2177-R,...,27/03/2018,F,IMPERADOR BAXTER VOLTA FRIA,IONA 777 SUPERSTITION SJ LALU,5/8HOL 3/8GIR,666,NaN,NaN,GIRO977401,977401
2,10639,LANAA FIV AFTERSHOCK BON.BELEM,12,9592-AG,MS ATLEES SHT AFTERSHOCK - ET,10,AX129928,JOANE FIV KUBERA,11,ACFG1840,...,22/12/2014,F,MS ATLEES SHT AFTERSHOCK - ET,JOANE FIV KUBERA,1/2HOL 1/2GIR,519,NaN,ACFG1840,207142,1086106
3,5191,MANDALA AFTERSHOCK 2K,12,4830-AR,MS ATLEES SHT AFTERSHOCK - ET,10,AX129928,GRACIOSA,11,APKK9,...,25/10/2016,F,MS ATLEES SHT AFTERSHOCK - ET,GRACIOSA,1/2HOL 1/2GIR,519,NaN,APKK9,207142,1086106
4,5407,BEATRIZ AFTERSHOCK DA CACHOEI,12,7058-AT,MS ATLEES SHT AFTERSHOCK - ET,10,AX129928,FLAMA VILAREJO FIV,11,HFV16,...,30/07/2016,F,MS ATLEES SHT AFTERSHOCK - ET,FLAMA VILAREJO FIV,1/2HOL 1/2GIR,519,NaN,HFV16,207142,1086106
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
433,6029,GRAVINA FEBO FR RECREIO,5858,9073-AS,FEBO FR RECREIO,58,6300-N,DORINA FR RECREIO,58,0860-T,...,13/06/2016,F,FEBO FR RECREIO,DORINA FR RECREIO,5/8HOL 3/8GIR,666,NaN,NaN,GIRO1114289,1114289
434,6029,GERMANA FEBO FR RECREIO,5858,9069-AS,FEBO FR RECREIO,58,6300-N,CAMPANHA FR RECREIO,58,6180-N,...,16/07/2016,F,FEBO FR RECREIO,CAMPANHA FR RECREIO,5/8HOL 3/8GIR,666,NaN,NaN,GIRO1114289,1114289
435,2381,ESTIVA QUIZAR JM NOVO HORIZONT,12,9958-AS,COLLEM FEVER QUIZAR,10,AX134708,SC MANCHETE PALADINO,11,MJJS411,...,16/05/2016,F,COLLEM FEVER QUIZAR,SC MANCHETE PALADINO,1/2HOL 1/2GIR,519,NaN,MJJS411,GIRO1481509,1481509
436,2381,DALIA I FIV BRADLEY JM NOVO HO,12,4906-AZ,ROCKALLI BRADLEY,10,AX117720,DALIA NOVO HORIZONTE,11,JMCH76,...,19/08/2017,F,ROCKALLI BRADLEY,DALIA NOVO HORIZONTE,1/2HOL 1/2GIR,519,NaN,JMCH76,210342,759942


# Buscando o id_produto na genealogia embrapa , que vira id_touro no df_todas_as_colunas pelo cod_pai

In [33]:
df_id_touro = pd.merge(df_todas_colunas, df_genealogia_embrapa[['id_produto', 'cod_prod']], left_on=['cod_pai'], right_on=['cod_prod'], suffixes=('', '_drop'))
df_id_touro

,reb_giro,nomea,COMPRAC,RGVACA,nomep,GSPAI,RGPAI,nomem,GSMAE,RGMAE,...,AN,NOMEVACA,DNASC,SEXO,NOMEPAI,NOMEMAE,cgen_a,comb,id_produto_drop,cod_prod_drop
0,6039,EDILAINE IMPERIO FIV 4303 JARD,12,AADA0352,IMPERIO TE STA EDWIGES,11,RIG163,SERTEC QATAR FABIANA PROGRESS,10,HBB/B-291392,...,2013,EDILAINE IMPERIO FIV 4303 JARDIM DO TURVO,18/10/2013,F,IMPERIO TE STA EDWIGES,SERTEC QATAR FABIANA PROGRESS,1/2HOL 1/2GIR,519,RIG163,4320031
1,73119,CELESTE 136 TEATRO FIV COOPERR,12,1239-BF,TEATRO DA SILVANIA,11,EFC383,LAGOS ASTRE QUIMICA 795 TE,10,BX335536,...,2016,CELESTE 136 TEATRO FIV COOPERRITA,05/02/2016,F,TEATRO DA SILVANIA,LAGOS ASTRE QUIMICA 795 TE,1/2HOL 1/2GIR,519,EFC383,3540404
2,3304,LAIZ TEATRO FIV ORIENTAL,12,4367-AS,TEATRO DA SILVANIA,11,EFC383,TUKINHA ALPA LEE-FIV,10,BX416793,...,2016,LAIZ TEATRO FIV ORIENTAL,22/04/2016,F,TEATRO DA SILVANIA,TUKINHA ALPA LEE-FIV,1/2HOL 1/2GIR,519,EFC383,3540404
3,11818,TANDARA TEATRO 1452 BARAO DA B,12,1091-BF,TEATRO DA SILVANIA,11,EFC383,BARAO D'BOCAINA TINA TOYSTORY,10,HBB/B-496454,...,2018,TANDARA TEATRO 1452 BARAO DA BOCAINA,11/05/2018,F,TEATRO DA SILVANIA,BARAO D'BOCAINA TINA TOYSTORY,1/2HOL 1/2GIR,519,EFC383,3540404
4,8689,ABDALLA TEATRO OASIS DA DIVISA,12,7293-BJ,TEATRO DA SILVANIA,11,EFC383,OASIS DA DIVISA ATINA ELEGANCE,10,BX468447,...,2017,ABDALLA TEATRO OASIS DA DIVISA,11/06/2017,F,TEATRO DA SILVANIA,OASIS DA DIVISA ATINA ELEGANCE,1/2HOL 1/2GIR,519,EFC383,3540404
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160,2381,GENIAL TPGL1889 JM NOVO HORIZO,38,2833-BF,KAREL FIV DA SALOBO,11,ABP1685,DUCHA BOLDWIN JM NOVO HORIZONT,34,4386-AN,...,2018,GENIAL TPGL1889 JM NOVO HORIZONTE,31/05/2018,F,KAREL FIV DA SALOBO,DUCHA BOLDWIN JM NOVO HORIZONT,3/8HOL 5/8GIR,417,ABP1685,12296894
161,9414,CETONA DA FRONTEIRA,14,4886-BW,PH DESTINO,11,PHPO456,ULTRA CANADA AFTERSHOCK FIV,12,2474-AH,...,2018,CETONA DA FRONTEIRA,14/11/2018,F,PH DESTINO,ULTRA CANADA AFTERSHOCK FIV,1/4HOL 3/4GIR,315,PHPO456,10323412
162,5569,DEMY CONAN MAO DA ANGOLINHA,14,4804-BE,CONAN ALBOS,11,SJTG93,SHRTC BAGDA 69 UERICA FARDO,12,4161-Z,...,2018,DEMY CONAN MAO DA ANGOLINHA,25/07/2018,F,CONAN ALBOS,SHRTC BAGDA 69 UERICA FARDO,1/4HOL 3/4GIR,315,SJTG93,13454046
163,5678,GBEZ 6056 GOMEATINGA,14,4255-AS,MILIONARIO SILVANIA,11,EFC1430,GBEZ 3176 GOMEATINGA,12,7419-AH,...,2016,GBEZ 6056 GOMEATINGA,24/10/2016,F,MILIONARIO SILVANIA,GBEZ 3176 GOMEATINGA,1/4HOL 3/4GIR,315,EFC1430,12110796


# Adicionando o id_touro no dataframe com todas as colunas pelo id_animal

In [34]:
df_todas_colunas = pd.merge(df_todas_colunas, df_id_touro[['IDANIMAL','id_produto_drop']], on=['IDANIMAL'], how='left')
df_todas_colunas = df_todas_colunas.drop_duplicates()
df_todas_colunas = df_todas_colunas.rename(columns={'id_produto_drop':'id_touro'})
df_todas_colunas

,reb_giro,nomea,COMPRAC,RGVACA,nomep,GSPAI,RGPAI,nomem,GSMAE,RGMAE,...,id_produto_y,AN,NOMEVACA,DNASC,SEXO,NOMEPAI,NOMEMAE,cgen_a,comb,id_touro
0,7461,NANA IMPERADOR BAXTER RF SAO F,5858,6353-AH,IMPERADOR BAXTER VOLTA FRIA,58,1459,JURITI HERBY RF SAO FRANCISCO,58,6877-N,...,NaN,2015,NANA IMPERADOR BAXTER RF SAO FRANCISCO,11/03/2015,F,IMPERADOR BAXTER VOLTA FRIA,JURITI HERBY RF SAO FRANCISCO,5/8HOL 3/8GIR,666,NaN
1,10639,LANAA FIV AFTERSHOCK BON.BELEM,12,9592-AG,MS ATLEES SHT AFTERSHOCK - ET,10,AX129928,JOANE FIV KUBERA,11,ACFG1840,...,NaN,2014,LANCA FIV AFTERSHOCK BON.BELEM,22/12/2014,F,MS ATLEES SHT AFTERSHOCK - ET,JOANE FIV KUBERA,1/2HOL 1/2GIR,519,NaN
2,6039,EDILAINE IMPERIO FIV 4303 JARD,12,AADA0352,IMPERIO TE STA EDWIGES,11,RIG163,SERTEC QATAR FABIANA PROGRESS,10,HBB/B-291392,...,NaN,2013,EDILAINE IMPERIO FIV 4303 JARDIM DO TURVO,18/10/2013,F,IMPERIO TE STA EDWIGES,SERTEC QATAR FABIANA PROGRESS,1/2HOL 1/2GIR,519,RIG163
3,1945,ARMA MERIDIAN DA MBL,12,1025-AN,SULLY HART MERIDIAN-ET,10,AX138576,FAMA,11,JNR167,...,NaN,2015,ARMA MERIDIAN DA MBL,27/04/2015,F,SULLY HART MERIDIAN-ET,FAMA,1/2HOL 1/2GIR,519,NaN
4,73119,CELESTE 136 TEATRO FIV COOPERR,12,1239-BF,TEATRO DA SILVANIA,11,EFC383,LAGOS ASTRE QUIMICA 795 TE,10,BX335536,...,NaN,2016,CELESTE 136 TEATRO FIV COOPERRITA,05/02/2016,F,TEATRO DA SILVANIA,LAGOS ASTRE QUIMICA 795 TE,1/2HOL 1/2GIR,519,EFC383
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
572,1945,JPZ QUIRERA SHAMROCK ASTER FIV,12,4032-Z,LADYS-MANOR PL SHAMROCK-ET,10,AX135714,JPZ ASTER FIV,11,JPZ64,...,NaN,2014,JPZ QUIRERA SHAMROCK ASTER FIV,18/02/2014,F,LADYS-MANOR PL SHAMROCK-ET,JPZ ASTER FIV,1/2HOL 1/2GIR,519,NaN
573,2381,DALIA I FIV BRADLEY JM NOVO HO,12,4906-AZ,ROCKALLI BRADLEY,10,AX117720,DALIA NOVO HORIZONTE,11,JMCH76,...,NaN,2017,DALIA I FIV BRADLEY JM NOVO HORIZONTE,19/08/2017,F,ROCKALLI BRADLEY,DALIA NOVO HORIZONTE,1/2HOL 1/2GIR,519,NaN
574,132,QUINTA I COLE AGM TRIUNFO,34,9243-AG,CO-OP TOYSTORY COLE-ET,10,AX129567,QUINTA FIV M,12,0473-O,...,NaN,2015,QUINTA I COLE AGM TRIUNFO,25/04/2015,F,CO-OP TOYSTORY COLE-ET,QUINTA FIV M,3/4HOL 1/4GIR,795,NaN
575,6915,LIA DAS ARABIAS,5858,6125-BR,TURBO DAS ARABIAS,5858,0079,LILIKA DAS ARABIAS,58,4916-AW,...,NaN,2019,LIA DAS ARABIAS,13/06/2019,F,TURBO DAS ARABIAS,LILIKA DAS ARABIAS,5/8HOL 3/8GIR,666,NaN


# Buscando o id_produto na genealogia embrapa , que vira id_vaca no df_todas_as_colunas pelo cod_mae

In [35]:
df_todas_colunas
df_id_vaca = pd.merge(df_todas_colunas, df_genealogia_embrapa[['id_produto', 'cod_prod']], left_on=['cod_mae'], right_on=['cod_prod'], indicator=True, suffixes=('', '_drop'))
df_id_vaca

,reb_giro,nomea,COMPRAC,RGVACA,nomep,GSPAI,RGPAI,nomem,GSMAE,RGMAE,...,DNASC,SEXO,NOMEPAI,NOMEMAE,cgen_a,comb,id_touro,id_produto_drop,cod_prod_drop,_merge
0,10639,LANAA FIV AFTERSHOCK BON.BELEM,12,9592-AG,MS ATLEES SHT AFTERSHOCK - ET,10,AX129928,JOANE FIV KUBERA,11,ACFG1840,...,22/12/2014,F,MS ATLEES SHT AFTERSHOCK - ET,JOANE FIV KUBERA,1/2HOL 1/2GIR,519,NaN,ACFG1840,9513273,both
1,1945,ARMA MERIDIAN DA MBL,12,1025-AN,SULLY HART MERIDIAN-ET,10,AX138576,FAMA,11,JNR167,...,27/04/2015,F,SULLY HART MERIDIAN-ET,FAMA,1/2HOL 1/2GIR,519,NaN,JNR167,10621711,both
2,83436,PURA FIV DOORMAN SJ LALU,12,2678-BK,VAL-BISSON DOORMAN,10,AX140558,KANETA DP,11,DPJ955,...,29/08/2019,F,VAL-BISSON DOORMAN,KANETA DP,1/2HOL 1/2GIR,519,NaN,DPJ955,12718455,both
3,83436,PALMIRA FIV BRYANT SJ LALU,12,2669-BK,WA-DEL BRYANT-ET,10,AX143718,KANETA DP,11,DPJ955,...,05/05/2019,F,WA-DEL BRYANT-ET,KANETA DP,1/2HOL 1/2GIR,519,NaN,DPJ955,12718455,both
4,83436,PANDORA FIV DOORMAN SJ LALU,12,2660-BK,VAL-BISSON DOORMAN,10,AX140558,KANETA DP,11,DPJ955,...,30/08/2019,F,VAL-BISSON DOORMAN,KANETA DP,1/2HOL 1/2GIR,519,NaN,DPJ955,12718455,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,9414,ULTRA GANGUE GILLESPY FIV,12,4887-BW,DE-SU GILLESPY-ET,10,AX138026,ULTRA DARA JAGUAR FIV,11,UTRG9,...,18/02/2019,F,DE-SU GILLESPY-ET,ULTRA DARA JAGUAR FIV,1/2HOL 1/2GIR,519,NaN,UTRG9,15225065,both
145,3717,MACIEIRA FIV VISTA ALEGRE,12,5569-BB,SEAGULL-BAY SUPERSIRE-ET,10,AX139812,ALBA COCHO D'AGUA,11,LMT15,...,07/02/2018,F,SEAGULL-BAY SUPERSIRE-ET,ALBA COCHO D'AGUA,1/2HOL 1/2GIR,519,NaN,LMT15,8745168,both
146,11818,FABULOSA FIV BRADNICK DA JJPZ,12,3852-BN,REGANCREST-GV S BRADNICK- ET,10,AX134761,FABIA FIV,11,UDI823,...,27/05/2019,F,REGANCREST-GV S BRADNICK- ET,FABIA FIV,1/2HOL 1/2GIR,519,NaN,UDI823,13327464,both
147,2381,ESTIVA QUIZAR JM NOVO HORIZONT,12,9958-AS,COLLEM FEVER QUIZAR,10,AX134708,SC MANCHETE PALADINO,11,MJJS411,...,16/05/2016,F,COLLEM FEVER QUIZAR,SC MANCHETE PALADINO,1/2HOL 1/2GIR,519,NaN,MJJS411,4220695,both


# Adicionando id_vaca no dataframe com todas as colunas

In [36]:
df_todas_colunas = pd.merge(df_todas_colunas, df_id_vaca[['IDANIMAL','id_produto_drop']], on=['IDANIMAL'], how='left')
df_todas_colunas = df_todas_colunas.drop_duplicates()
df_todas_colunas = df_todas_colunas.rename(columns={'id_produto_drop':'id_vaca'})
df_todas_colunas

,reb_giro,nomea,COMPRAC,RGVACA,nomep,GSPAI,RGPAI,nomem,GSMAE,RGMAE,...,AN,NOMEVACA,DNASC,SEXO,NOMEPAI,NOMEMAE,cgen_a,comb,id_touro,id_vaca
0,7461,NANA IMPERADOR BAXTER RF SAO F,5858,6353-AH,IMPERADOR BAXTER VOLTA FRIA,58,1459,JURITI HERBY RF SAO FRANCISCO,58,6877-N,...,2015,NANA IMPERADOR BAXTER RF SAO FRANCISCO,11/03/2015,F,IMPERADOR BAXTER VOLTA FRIA,JURITI HERBY RF SAO FRANCISCO,5/8HOL 3/8GIR,666,NaN,NaN
1,10639,LANAA FIV AFTERSHOCK BON.BELEM,12,9592-AG,MS ATLEES SHT AFTERSHOCK - ET,10,AX129928,JOANE FIV KUBERA,11,ACFG1840,...,2014,LANCA FIV AFTERSHOCK BON.BELEM,22/12/2014,F,MS ATLEES SHT AFTERSHOCK - ET,JOANE FIV KUBERA,1/2HOL 1/2GIR,519,NaN,ACFG1840
2,6039,EDILAINE IMPERIO FIV 4303 JARD,12,AADA0352,IMPERIO TE STA EDWIGES,11,RIG163,SERTEC QATAR FABIANA PROGRESS,10,HBB/B-291392,...,2013,EDILAINE IMPERIO FIV 4303 JARDIM DO TURVO,18/10/2013,F,IMPERIO TE STA EDWIGES,SERTEC QATAR FABIANA PROGRESS,1/2HOL 1/2GIR,519,RIG163,NaN
3,1945,ARMA MERIDIAN DA MBL,12,1025-AN,SULLY HART MERIDIAN-ET,10,AX138576,FAMA,11,JNR167,...,2015,ARMA MERIDIAN DA MBL,27/04/2015,F,SULLY HART MERIDIAN-ET,FAMA,1/2HOL 1/2GIR,519,NaN,JNR167
4,73119,CELESTE 136 TEATRO FIV COOPERR,12,1239-BF,TEATRO DA SILVANIA,11,EFC383,LAGOS ASTRE QUIMICA 795 TE,10,BX335536,...,2016,CELESTE 136 TEATRO FIV COOPERRITA,05/02/2016,F,TEATRO DA SILVANIA,LAGOS ASTRE QUIMICA 795 TE,1/2HOL 1/2GIR,519,EFC383,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,1945,JPZ QUIRERA SHAMROCK ASTER FIV,12,4032-Z,LADYS-MANOR PL SHAMROCK-ET,10,AX135714,JPZ ASTER FIV,11,JPZ64,...,2014,JPZ QUIRERA SHAMROCK ASTER FIV,18/02/2014,F,LADYS-MANOR PL SHAMROCK-ET,JPZ ASTER FIV,1/2HOL 1/2GIR,519,NaN,JPZ64
565,2381,DALIA I FIV BRADLEY JM NOVO HO,12,4906-AZ,ROCKALLI BRADLEY,10,AX117720,DALIA NOVO HORIZONTE,11,JMCH76,...,2017,DALIA I FIV BRADLEY JM NOVO HORIZONTE,19/08/2017,F,ROCKALLI BRADLEY,DALIA NOVO HORIZONTE,1/2HOL 1/2GIR,519,NaN,JMCH76
566,132,QUINTA I COLE AGM TRIUNFO,34,9243-AG,CO-OP TOYSTORY COLE-ET,10,AX129567,QUINTA FIV M,12,0473-O,...,2015,QUINTA I COLE AGM TRIUNFO,25/04/2015,F,CO-OP TOYSTORY COLE-ET,QUINTA FIV M,3/4HOL 1/4GIR,795,NaN,NaN
567,6915,LIA DAS ARABIAS,5858,6125-BR,TURBO DAS ARABIAS,5858,0079,LILIKA DAS ARABIAS,58,4916-AW,...,2019,LIA DAS ARABIAS,13/06/2019,F,TURBO DAS ARABIAS,LILIKA DAS ARABIAS,5/8HOL 3/8GIR,666,NaN,NaN


# Pegando o id_produto que vira id_touro dos touros_gir na genealogia embrapa, em que o registro comeca com 3 letras

In [40]:
# somente gs_pai=11 e gs = 0 na genealogia
df_sub_tabela_gir = df_todas_colunas[df_todas_colunas['GSPAI']==11]
df_sub_tabela_gir = df_sub_tabela_gir[df_sub_tabela_gir['RGPAI'].apply(lambda x: bool(re.match(r'^[a-zA-Z]{3}', x)))]
df_sub_tabela_gir

,reb_giro,nomea,COMPRAC,RGVACA,nomep,GSPAI,RGPAI,nomem,GSMAE,RGMAE,...,AN,NOMEVACA,DNASC,SEXO,NOMEPAI,NOMEMAE,cgen_a,comb,id_touro,id_vaca
2,6039,EDILAINE IMPERIO FIV 4303 JARD,12,AADA0352,IMPERIO TE STA EDWIGES,11,RIG163,SERTEC QATAR FABIANA PROGRESS,10,HBB/B-291392,...,2013,EDILAINE IMPERIO FIV 4303 JARDIM DO TURVO,18/10/2013,F,IMPERIO TE STA EDWIGES,SERTEC QATAR FABIANA PROGRESS,1/2HOL 1/2GIR,519,RIG163,NaN
4,73119,CELESTE 136 TEATRO FIV COOPERR,12,1239-BF,TEATRO DA SILVANIA,11,EFC383,LAGOS ASTRE QUIMICA 795 TE,10,BX335536,...,2016,CELESTE 136 TEATRO FIV COOPERRITA,05/02/2016,F,TEATRO DA SILVANIA,LAGOS ASTRE QUIMICA 795 TE,1/2HOL 1/2GIR,519,EFC383,NaN
5,10586,JHV CANASTRA ENLEVO,12,5011-BG,ENLEVO SILVANIA,11,EFC717,JHV FORMATURA 315,10,C-3602,...,2019,JHV CANASTRA ENLEVO,18/06/2019,F,ENLEVO SILVANIA,JHV FORMATURA 315,1/2HOL 1/2GIR,519,EFC717,NaN
8,22541,KELLY ESPELHO JF DA SANTA RITA,12,7974-AM,ESPELHO TE DE BRASILIA,11,RRP5664,RACKELY JF DA SANTA RITA,10,A-7744,...,2012,KELLY ESPELHO JF DA SANTA RITA,06/04/2012,F,ESPELHO TE DE BRASILIA,RACKELY JF DA SANTA RITA,1/2HOL 1/2GIR,519,RRP5664,NaN
11,8490,REUNIDAS HD FLORIANA 17081 FEN,14,8038-BT,FB FENIX,11,FBGO681,REUNIDAS HD DADIVA 15897 HALEY,12,4322-AN,...,2018,REUNIDAS HD FLORIANA 17081 FENIX,01/03/2018,F,FB FENIX,REUNIDAS HD DADIVA 15897 HALEY,1/4HOL 3/4GIR,315,FBGO681,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
545,5191,NOTICIA ENLEVO 2K,12,1662-AY,ENLEVO SILVANIA,11,EFC717,2K HARPA 483,10,BR1717008,...,2017,NOTICIA ENLEVO 2K,14/04/2017,F,ENLEVO SILVANIA,2K HARPA 483,1/2HOL 1/2GIR,519,EFC717,NaN
547,9000,GALEGA GALEGO DA COLI,14,8985-AN,GALEGO FIV F. MUTUM,11,MUT899,BOLIVIA ARMSTEAD DA COLI,12,4041-U,...,2014,GALEGA GALEGO DA COLI,20/05/2014,F,GALEGO FIV F. MUTUM,BOLIVIA ARMSTEAD DA COLI,1/4HOL 3/4GIR,315,MUT899,NaN
551,7461,PITANGA RF SAO FRANCISCO,38,4055-AX,CAXAMBU FZD LUMI,11,FZLM215,HUMORISTA ZESTY RF SAO FRANCIS,34,5543-E,...,2017,PITANGA RF SAO FRANCISCO,30/04/2017,F,CAXAMBU FZD LUMI,HUMORISTA ZESTY RF SAO FRANCIS,3/8HOL 5/8GIR,417,FZLM215,NaN
555,10586,JHV BRAUNA FIV GENGIS KHAN,12,6584-BF,GENGIS KHAN DE BRASILIA,11,RRP6097,BRAUNAS GOLDWYN NARITA LARA-TE,10,HBB/B-388117,...,2018,JHV BRAUNA FIV GENGIS KHAN,24/09/2018,F,GENGIS KHAN DE BRASILIA,BRAUNAS GOLDWYN NARITA LARA-TE,1/2HOL 1/2GIR,519,RRP6097,NaN


# Pegando os id_produto's dos touros_gir na genealogia embrapa, em que o registro nao comeca com 3 letras

In [47]:
df_sub_tabela_gir1 = df_todas_colunas[df_todas_colunas['GSPAI']==11]
df_sub_tabela_gir1 = df_sub_tabela_gir1[~df_sub_tabela_gir1['RGPAI'].apply(lambda x: bool(re.match(r'^[a-zA-Z]{3}', x)))]
#Pegando os 4 primeiros caracteres do nome
df_sub_tabela_gir1['nome4caracteres'] = df_sub_tabela_gir1['nomep'].str.slice(start=0, stop=4)


df_subtabela_genealogia = df_genealogia_embrapa
df_subtabela_genealogia = df_subtabela_genealogia[df_subtabela_genealogia['gs']==0]
df_subtabela_genealogia = df_subtabela_genealogia[df_subtabela_genealogia['sexo']=='M']
df_subtabela_genealogia = df_subtabela_genealogia[~df_subtabela_genealogia['registro'].isnull()]
df_subtabela_genealogia = df_subtabela_genealogia[~df_subtabela_genealogia['nome'].isnull()]
df_subtabela_genealogia = df_subtabela_genealogia[~df_subtabela_genealogia['registro'].apply(lambda x: bool(re.match(r'^[a-zA-Z]{3}', x)))]
df_subtabela_genealogia['nome4caracteres'] = df_subtabela_genealogia['nome'].str.slice(start=0, stop=4)

dj_juntando_tabelas = pd.merge(df_sub_tabela_gir1, df_subtabela_genealogia[['id_produto', 'registro', 'nome4caracteres']], left_on=['nome4caracteres', 'RGPAI'], right_on=['nome4caracteres', 'registro'])
dj_juntando_tabelas

,reb_giro,nomea,COMPRAC,RGVACA,nomep,GSPAI,RGPAI,nomem,GSMAE,RGMAE,...,SEXO,NOMEPAI,NOMEMAE,cgen_a,comb,id_touro,id_vaca,nome4caracteres,id_produto_y,registro
0,5191,MENINA MODELO 2K,12,4833-AR,MODELO TE DE BRASILIA,11,B5213,FEITICEIRA 2K,10,C-1072,...,F,MODELO TE DE BRASILIA,FEITICEIRA 2K,1/2HOL 1/2GIR,519,106183,NaN,MODE,106183,B5213
1,5191,MELISSA MODELO 2K,12,5416-AR,MODELO TE DE BRASILIA,11,B5213,IOGA 2K,10,C-1073,...,F,MODELO TE DE BRASILIA,IOGA 2K,1/2HOL 1/2GIR,519,106183,NaN,MODE,106183,B5213
2,1945,BROMELIA FIV DO JRD,11,JRDG28,RADAR DOS POCOES,11,A7368,PROFANA 6,11,EBQG1,...,F,RADAR DOS POCOES,PROFANA 6,GIR,11,103136,EBQG1,RADA,103136,A7368
3,5407,GARFIEIRA,11,PVBG1140,RADAR DOS POCOES,11,A7368,LARA FIV DA SADONANA,11,SDNA34,...,F,RADAR DOS POCOES,LARA FIV DA SADONANA,GIR,11,103136,SDNA34,RADA,103136,A7368
4,5407,FORTUNA DA CRISTAL,11,PVBG855,RADAR DOS POCOES,11,A7368,RADULA TE DO GAVIAO,11,GAV980,...,F,RADAR DOS POCOES,RADULA TE DO GAVIAO,GIR,11,103136,GAV980,RADA,103136,A7368
5,8030,VENTOSA CADARSO FZD,14,4241-AQ,CADARSO C-054,11,B32,SAVANA PLANET FIV FZD,12,4961-U,...,F,CADARSO C-054,SAVANA PLANET FIV FZD,1/4HOL 3/4GIR,315,NaN,NaN,CADA,102812,B32


# Adicionando os id_produto's  encontrados

# Pegando os id_produto’s das vacas na genealogia embrapa, em que o registro nao comeca com 3 letras

In [45]:
df_sub_tabela_gir1 = df_todas_colunas[df_todas_colunas['GSMAE']==11]
df_sub_tabela_gir1 = df_sub_tabela_gir1[~df_sub_tabela_gir1['RGMAE'].apply(lambda x: bool(re.match(r'^[a-zA-Z]{3}', x)))]
#Pegando os 4 primeiros caracteres do nome
df_sub_tabela_gir1['nome4caracteres'] = df_sub_tabela_gir1['nomem'].str.slice(start=0, stop=4)


df_subtabela_genealogia1 = df_genealogia_embrapa
df_subtabela_genealogia1 = df_subtabela_genealogia1[df_subtabela_genealogia1['gs']==0]
df_subtabela_genealogia1 = df_subtabela_genealogia1[df_subtabela_genealogia1['sexo']=='F']
df_subtabela_genealogia1 = df_subtabela_genealogia1[~df_subtabela_genealogia1['registro'].isnull()]
df_subtabela_genealogia1 = df_subtabela_genealogia1[~df_subtabela_genealogia1['nome'].isnull()]
df_subtabela_genealogia1 = df_subtabela_genealogia1[~df_subtabela_genealogia1['registro'].apply(lambda x: bool(re.match(r'^[a-zA-Z]{3}', x)))]
df_subtabela_genealogia1['nome4caracteres'] = df_subtabela_genealogia1['nome'].str.slice(start=0, stop=4)

dj_juntando_tabelas = pd.merge(df_sub_tabela_gir1, df_subtabela_genealogia1[['id_produto', 'registro', 'nome4caracteres']], left_on=['nome4caracteres', 'RGMAE'], right_on=['nome4caracteres', 'registro'])
dj_juntando_tabelas

,id_produto,id,abcz,registro,nome,sexo,paternidade,ck_sire,id_touro,id_vaca,...,rebc2_pr,cat,ficha_a,cod_prod,IDANIMAL,origem,registro_pai,registro_mae,AN,nome10caracteres
0,0001,0001,None,None,nan,M,None,None,None,None,...,99999,None,0,0,0,OUTRAS,None,None,1982,nan
1,0003,0003,None,None,nan,M,None,None,None,None,...,99999,None,0,0,0,OUTRAS,None,None,1968,nan
2,0006,0006,None,None,nan,M,None,None,None,None,...,99999,None,0,0,0,OUTRAS,None,None,1967,nan
4,03,03,None,None,nan,M,None,None,None,None,...,99999,None,0,0,0,OUTRAS,None,None,1965,nan
5,06,06,None,None,nan,M,None,None,None,None,...,99999,None,0,0,0,OUTRAS,None,None,1976,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
463539,ZZZ3,ZZZ3,S,ZZZ3,ABUSO,M,None,None,KCA472,132091,...,7145-ABCZ 1,PO,0,3155995,0,ABCZ,KCA472,KB10,1998,ABUSO
463540,ZZZ4,ZZZ4,S,ZZZ4,ALIBI,M,None,None,131870,131845,...,7145-ABCZ 1,PO,0,3158056,0,ABCZ,K170,KA7372,1998,ALIBI
463542,ZZZ6,ZZZ6,S,ZZZ6,ABONADO,M,None,None,131839,131947,...,7145-ABCZ 1,PO,0,3156002,0,ABCZ,K42,KA9413,1998,ABONADO
463544,ZZZ8,ZZZ8,S,ZZZ8,AGASALHO,M,None,None,121118,131985,...,7145-ABCZ 1,PO,0,3247323,0,ABCZ,K830,KA7368,1998,AGASALHO


In [ ]:
df_id_touro = pd.merge(df_todas_colunas, df_genealogia_embrapa[['registro', 'id_produto']], left_on=['RGPAI'], right_on=['registro'], suffixes=('', '_drop'))
df_id_touro = df_id_touro.drop_duplicates()
df_id_touro

In [22]:
df_genealogia_embrapa.columns

Index(['id_produto', 'id', 'abcz', 'registro', 'nome', 'sexo', 'paternidade',
       'ck_sire', 'id_touro', 'id_vaca', 'ck_dam', 'gen_valido', 'nasc',
       'nasc_est', 'cgen_a', 'gs', 'gr_psg', 'gr_ps', 'gr_tp', 'rebc2_cri',
       'rebc2_pr', 'cat', 'ficha_a', 'cod_prod', 'IDANIMAL', 'origem',
       'registro_pai', 'registro_mae', 'AN', 'nome10caracteres'],
      dtype='object')

In [39]:
df_todas_colunas.columns

Index(['reb_giro', 'nomea', 'COMPRAC', 'RGVACA', 'nomep', 'GSPAI', 'RGPAI',
       'nomem', 'GSMAE', 'RGMAE', 'DNV', 'MNV', 'ANV', 'UF', 'OP', 'DP', 'MP',
       'AP', 'DURLAC', 'LTOT', 'L365', 'L305', 'DE', 'ME', 'AE', 'CAUSA',
       'INTER', 'IDPAI', 'IDMAE', 'IDANIMAL', 'TORNEIO', 'RA', 'TIPO',
       'QTD_ORDENHAS', 'nasc', 'parto', 'secagem', 'anopc', 'anop_menos1c',
       'rebc', 'rac', 'nordc', 'rebanoe', 'ordem', 'gs', 'sexo', 'cod_prod',
       'cod_pai', 'cod_mae', 'reb', 'id_produto', 'nome10caracteres',
       'id_produto_x', 'id_produto_y', 'AN', 'NOMEVACA', 'DNASC', 'SEXO',
       'NOMEPAI', 'NOMEMAE', 'cgen_a', 'comb', 'id_touro', 'id_vaca'],
      dtype='object')

In [ ]:
df_id_touro = pd.merge(df_todas_colunas, df_genealogia_embrapa[['registro_pai', 'cod_prod']], left_on=['RGPAI'], right_on=['registro'], how='left', suffixes=('', '_drop'))
df_id_touro = df_id_touro.drop_duplicates()
df_id_touro